# 기사(헤드라인) 전문가 에이전트

사용자 입력 일자를 기준으로 해당 일자 포함 과거 7일간의 기사 헤드라인 CSV 데이터를 참고해, 시장 분위기·이슈를 해석하고 매수/매도 관점의 구조화된 의견을 제시합니다.

- **입력**: 평가하고 싶은 일자 (예: 2018-02-19)
- **출력**: Pydantic 스키마 (긍정적/보통/부정적, 시장 흐름 분석, 단기·장기 관점, 주목 이슈/기간)

## 1. 환경 설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

python-dotenv could not parse statement starting at line 20


True

In [3]:
# LangSmith 추적 (선택) https://smith.langchain.com
from langchain_teddynote import logging
logging.langsmith("article-expert-agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
article-expert-agent


In [4]:
import pandas as pd
from pathlib import Path
from typing import Literal, List, Dict

from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

## 2. 데이터 로드

CSV 경로는 상수로 두어 추후 API로 교체 시 `load_article_data()`만 수정하면 됩니다.

In [5]:
def get_csv_path() -> Path:
    """노트북과 같은 디렉터리의 CSV 경로 (추후 API로 교체 가능)."""
    return Path("article_headline.csv")

def load_article_data(csv_path: Path | None = None) -> pd.DataFrame:
    csv_path = csv_path or get_csv_path()
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()
    df["published_at"] = pd.to_datetime(df["published_at"])
    df["date"] = df["published_at"].dt.normalize()
    df = df.sort_values(["date", "published_at"]).reset_index(drop=True)
    return df

df = load_article_data()
df.head(10)

,headline,published_at,date
0,비트코인 재단 이사 “올 연말까지 비트코인 4만달러 예상..알트코인 90%는 실패”,2018-02-19 07:05:00,2018-02-19
1,[굿모닝 비트코인] 0219 비트코인 1만선 회복…그 배경은?,2018-02-19 07:06:00,2018-02-19
2,[이더리움101]이더와 이더리움,2018-02-19 07:10:00,2018-02-19
3,"[이더리움101]이더리움, 어디서∙어떻게 살까요?",2018-02-19 07:11:00,2018-02-19
4,스마트 컨트랙트(Smart Contract)의 예,2018-02-19 07:11:00,2018-02-19
5,[이더리움101]이더리움과 Dapp,2018-02-19 07:12:00,2018-02-19
6,[코인 국내외 주요일정 – 2월19일],2018-02-19 10:09:00,2018-02-19
7,[이 시각 코인] 글로벌 암호화폐 거래량 순위(오전 10시 10분),2018-02-19 10:11:00,2018-02-19
8,[상승률 상위·하락률 상위 코인 – 2월19일],2018-02-19 10:23:00,2018-02-19
9,美 “규제 앞서 충분히 검토할 것”,2018-02-19 11:21:00,2018-02-19


In [6]:
MAX_HEADLINES_PER_DAY = 30

def get_period_data(
    df: pd.DataFrame,
    target_date: str,
    days_before: int = 7,
    max_headlines_per_day: int = MAX_HEADLINES_PER_DAY,
) -> str:
    """선택일 포함, 선택일 기준 과거 days_before일까지의 헤드라인을 일자별로 묶어 반환."""
    target = pd.to_datetime(target_date).normalize()
    start = target - pd.Timedelta(days=days_before)
    end = target
    mask = (df["date"] >= start) & (df["date"] <= end)
    period = df.loc[mask, ["date", "headline", "published_at"]].copy()
    lines = []
    for d, g in period.groupby("date", sort=True):
        g = g.sort_values("published_at").head(max_headlines_per_day)
        headlines = g["headline"].astype(str).str.strip().tolist()
        day_str = pd.Timestamp(d).strftime("%Y-%m-%d")
        lines.append(day_str + ": " + " | ".join(headlines))
    return "\n".join(lines)

# 예시 (선택일 포함 과거 7일 = 총 8일)
get_period_data(df, "2019-02-19", days_before=7)[:1500]

'2019-02-12: 노보그라츠, 매크로 펀드 1%는 비트코인에 투자해야 | (표) CME 비트코인 선물 시세 (뉴욕시간 11일 오전 10시26분 기준)…2월물, 10달러 내린 3605달러 | 비트코인 4000달러 가시권 진입… 3700달러 저항선 돌파가 관건 | 2018 하반기 CME, CBOE 비트코인 선물 계약 크게 감소 – 트레이드블록 보고서 | [뉴욕 코인시황/장중] 전반적으로 소폭의 하락세 나타내…비트코인, 3650선 중심의 횡보세 | UAE 익스체인지와 유니모니, 리플 이용 국제 송금 서비스 가동 | [표] 이 시간 상승률/하락률 상위 10개 코인 (뉴욕시간 2월 11일 오후 1시18분 기준) | 비트코인 비판론자 크루그먼, 세계 경기 침체 경고 | LN의 스퀘어 캐시 앱 통합은 ‘만약’이 아닌 ‘언제’의 문제 – 잭 도시 | [표] 글로벌 암호화폐 거래소 거래량 톱10 (뉴욕시간 2월11일 오후 3시 기준) | “암호화폐 겨울” 속 거래소들 OTC로 활로 찾는다 – 코인텔레그래프 | IBM 블록체인+사물인터넷 가뭄 대처 협력 프로젝트 | 비트코인 선물과 국채 혼합 ETF 출시 위한 규정 개정안 SEC에 제출 | 크레딧 스위스와 방코 베스트, 블록체인 이용한 펀드 트랜잭션 성공 | 유럽의회 에바 카일리 “중앙은행이 블록체인 혜택 못 막아” | [뉴욕 코인시황/마감] 후장 들어 보합세 보여…비트코인, 3650선 중심의 횡보세 지속 | [굿모닝 비트코인] 0212 조정받는 비트코인, 혼조세 짙은 암호화폐 시장..대시 10% 급등 | [이 시각 코인] 글로벌 암호화폐&거래소 거래량 순위 (오전 7시 50분) | [뉴욕증시] 무역협상 결과 기다리며 ‘관망’ 모멘텀 실종 | 2월 12일, 간추린 간밤의 암호화폐 소식 | 후오비 코리아, 업데이트 된 암호화폐 거래 앱 배포 | 카카오스탁, 5년만에 누적거래액 53.3조..사용자 25~34세 남성 87%로 압도적 | 빗썸, UAE에 정부 공인 암호화폐 거래소 세운다 | 암호화폐 시장, 봄은 멀었다 | “정부,

## 3. Pydantic 응답 스키마 및 Parser

In [7]:
class NotablePeriod(BaseModel):
    title: str = Field(description="주목할 만한 이슈/기사 구간의 제목")
    period_text: str = Field(
        description="기간 문자열. 선택된 날짜 1일 + 이전 7일 (예: 2018-02-12 ~ 2018-02-19)"
    )
    period_data: List[Dict[str, str]] = Field(
        description='해당 기간의 대표 헤드라인 요약. 날짜(키)와 요약 또는 대표 헤드라인(값). 최대 7개. 예: [{"2018-02-17": "비트코인 1만선 회복 관련 보도"}]',
        max_length=7,
    )

class ArticleExpertResponse(BaseModel):
    verdict: Literal["긍정적", "보통", "부정적"] = Field(
        description="해당 일자 매수/매도 판단에 대한 종합 의견 (반드시 세 값 중 하나)"
    )
    market_flow_analysis: str = Field(
        description="해당 기간 시장 흐름 분석: 헤드라인에서 읽힌 시장·규제·심리 등 흐름 설명"
    )
    short_long_term_perspective: str = Field(
        description="단기(해당 일 전후 뉴스 톤)와 장기(해당 8일 구간이 시사하는 바) 관점"
    )
    notable_periods: List[NotablePeriod] = Field(
        description="주목할 만한 이슈/기사 구간과 그 요약 (항목 최대 5개, 각 기간 최대 7일)"
    )

parser = PydanticOutputParser(pydantic_object=ArticleExpertResponse)

## 4. 프롬프트, LLM, 체인

In [8]:
prompt = load_prompt("prompts/article_expert.yaml", encoding="utf-8")
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [9]:
MODEL_NAME = "gpt-4.1"
llm = ChatOpenAI(temperature=0, model=MODEL_NAME)
chain = prompt | llm | parser

## 5. 사용자 일자 입력 및 실행

In [10]:
target_date = "2019-02-19"
period_data = get_period_data(df, target_date, days_before=7)

In [11]:
response = chain.invoke({
    "target_date": target_date,
    "period_data": period_data,
})
response

ArticleExpertResponse(verdict='긍정적', market_flow_analysis="2019-02-19를 전후로 암호화폐 시장은 뚜렷한 반등세와 긍정적 심리가 부각되었습니다. 해당일 헤드라인에서는 비트코인 4000달러 돌파 시도, 이더리움·이오스 등 주요 알트코인 급등, 거래량 9개월 최고치 등 강한 매수세가 확인됩니다. 이더리움 하드포크 기대감, 비트메인 신제품 공개, 기관투자 장기투자 언급 등 업계 호재도 다수 포착됩니다. 다만 일부 부정적 이슈(비트메인 손실, 김치프리미엄 역전 등)도 있으나, 전반적으로 시장 분위기는 '해빙기' 기대와 함께 강한 상승 모멘텀을 보이고 있습니다.", short_long_term_perspective="단기적으로는 이더리움 하드포크와 주요 코인 급등, 거래량 증가 등으로 강한 매수세와 투자심리 개선이 뚜렷합니다. 전일~당일 헤드라인에서 '급등', '돌파', '해빙기' 등 긍정적 단어가 반복적으로 등장합니다. 장기적으로는 2월 초부터 이어진 박스권·약세장 이후, 점진적 반등과 기관투자, 블록체인 실사용 확대, 글로벌 규제 논의 등 구조적 개선 신호가 감지됩니다. 다만, 시장이 완전히 회복 국면에 진입했다고 단정하긴 이르나, 단기 모멘텀은 확실히 긍정적입니다.", notable_periods=[NotablePeriod(title='이더리움 하드포크 및 주요 코인 급등 구간', period_text='2019-02-18 ~ 2019-02-19', period_data=[{'2019-02-18': '비트코인·이더리움 등 박스권 속 이더리움 6~8% 급등, 하드포크 기대감'}, {'2019-02-19': '비트코인 4000달러 돌파 시도, 이더리움·이오스 등 급등, 거래량 9개월 최고치'}]), NotablePeriod(title='기관투자·블록체인 실사용 확대 및 규제 논의', period_text='2019-02-13 ~ 2019-02-16', period_data=[{'2019-02-13': '모간 크릭, 

In [12]:
print("=== 종합 의견 (verdict) ===")
print(response.verdict)
print("\n=== 시장 흐름 분석 ===")
print(response.market_flow_analysis)
print("\n=== 단기·장기 관점 ===")
print(response.short_long_term_perspective)
print("\n=== 주목 이슈 ===")
for p in response.notable_periods:
    print(f"  - title: {p.title}")
    print(f"    period_text: {p.period_text}")
    print(f"    period_data: {p.period_data}")

=== 종합 의견 (verdict) ===
긍정적

=== 시장 흐름 분석 ===
2019-02-19를 전후로 암호화폐 시장은 뚜렷한 반등세와 긍정적 심리가 부각되었습니다. 해당일 헤드라인에서는 비트코인 4000달러 돌파 시도, 이더리움·이오스 등 주요 알트코인 급등, 거래량 9개월 최고치 등 강한 매수세가 확인됩니다. 이더리움 하드포크 기대감, 비트메인 신제품 공개, 기관투자 장기투자 언급 등 업계 호재도 다수 포착됩니다. 다만 일부 부정적 이슈(비트메인 손실, 김치프리미엄 역전 등)도 있으나, 전반적으로 시장 분위기는 '해빙기' 기대와 함께 강한 상승 모멘텀을 보이고 있습니다.

=== 단기·장기 관점 ===
단기적으로는 이더리움 하드포크와 주요 코인 급등, 거래량 증가 등으로 강한 매수세와 투자심리 개선이 뚜렷합니다. 전일~당일 헤드라인에서 '급등', '돌파', '해빙기' 등 긍정적 단어가 반복적으로 등장합니다. 장기적으로는 2월 초부터 이어진 박스권·약세장 이후, 점진적 반등과 기관투자, 블록체인 실사용 확대, 글로벌 규제 논의 등 구조적 개선 신호가 감지됩니다. 다만, 시장이 완전히 회복 국면에 진입했다고 단정하긴 이르나, 단기 모멘텀은 확실히 긍정적입니다.

=== 주목 이슈 ===
  - title: 이더리움 하드포크 및 주요 코인 급등 구간
    period_text: 2019-02-18 ~ 2019-02-19
    period_data: [{'2019-02-18': '비트코인·이더리움 등 박스권 속 이더리움 6~8% 급등, 하드포크 기대감'}, {'2019-02-19': '비트코인 4000달러 돌파 시도, 이더리움·이오스 등 급등, 거래량 9개월 최고치'}]
  - title: 기관투자·블록체인 실사용 확대 및 규제 논의
    period_text: 2019-02-13 ~ 2019-02-16
    period_data: [{'2019-02-13': '모간 크릭, 공적 연금 투자 유치 성공 / 블록체인 기반 지역화폐 도입 발표'}, {'2